In [1]:
import pandas as pd, numpy as np, os
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
PATH ="D:/Tech Learning/Kaggle/US patent p2p matching/oof results/"
FILES = os.listdir(PATH)

OOF = np.sort( [f for f in FILES if 'oof' in f] )
OOF_CSV = [pd.read_csv(PATH+k) for k in OOF]

print('We have %i oof files...'%len(OOF))
print(); print(OOF)

We have 10 oof files...

['oof_1.01_full.csv' 'oof_1.07_full.csv' 'oof_1.12_full.csv'
 'oof_1.13_full.csv' 'oof_2.01_full.csv' 'oof_3.01_full.csv'
 'oof_3.02_full.csv' 'oof_5.01_full.csv' 'oof_5.02_full.csv'
 'oof_6.09_full.csv']


In [3]:
x = np.zeros(( len(OOF_CSV[0]),len(OOF) ))
for k in range(len(OOF)):
    OOF_CSV[k] = OOF_CSV[k].sort_values(by = "id")
    x[:,k] = OOF_CSV[k].pred.values
    
TRUE = OOF_CSV[0].score.values

In [4]:
TRUE

array([0.25, 0.5 , 0.25, ..., 0.  , 0.  , 0.5 ])

In [5]:
def get_score(y_true, y_pred):
    score = stats.pearsonr(y_true, y_pred)[0]
    return score

In [6]:
all = []
for k in range(x.shape[1]):
    pearson = get_score(OOF_CSV[0].score,x[:,k])
    all.append(pearson)
    print('Model %i has OOF pearson = %.4f'%(k,pearson))
    
m = [np.argmax(all)]; w = []

Model 0 has OOF pearson = 0.8225
Model 1 has OOF pearson = 0.8209
Model 2 has OOF pearson = 0.8186
Model 3 has OOF pearson = 0.8219
Model 4 has OOF pearson = 0.8310
Model 5 has OOF pearson = 0.8279
Model 6 has OOF pearson = 0.8281
Model 7 has OOF pearson = 0.8283
Model 8 has OOF pearson = 0.8258
Model 9 has OOF pearson = 0.8176


In [7]:
old = np.max(all); 

RES = 200; 
PATIENCE = 10; 
TOL = 0.0003
DUPLICATES = False

print('Ensemble Pearson = %.4f by beginning with model %i'%(old,m[0]))
print('Model %i is %s' %(m[0],OOF[m[0]]))
print()

for kk in range(len(OOF)):
    
    # BUILD CURRENT ENSEMBLE
    md = x[:,m[0]]
    for i,k in enumerate(m[1:]):
        md = w[i]*x[:,k] + (1-w[i])*md
        
    # FIND MODEL TO ADD
    mx = 0; mx_k = 0; mx_w = 0
    print('Searching for best model to add... ')
    
    # TRY ADDING EACH MODEL
    for k in range(x.shape[1]):
        print(k,', ',end='')
        if not DUPLICATES and (k in m): continue
            
        # EVALUATE ADDING MODEL K WITH WEIGHTS W
        bst_j = 0; bst = 0; ct = 0
        for j in range(RES):
            tmp = j/RES*x[:,k] + (1-j/RES)*md
            pearson = get_score(TRUE,tmp)
            if pearson>bst:
                bst = pearson
                bst_j = j/RES
            else: ct += 1
            if ct>PATIENCE: break
        if bst>mx:
            mx = bst
            mx_k = k
            mx_w = bst_j
            
    # STOP IF INCREASE IS LESS THAN TOL
    inc = mx-old
    if inc<=TOL: 
        print(); print('No increase. Stopping.')
        break
        
    # DISPLAY RESULTS
    print(); #print(kk,mx,mx_k,mx_w,'%.5f'%inc)
    print('Ensemble pearson = %.4f after adding model %i with weight %.3f. Increase of %.4f'%(mx,mx_k,mx_w,inc))
    print('Model %i is %s' %(mx_k,OOF[mx_k]))
    print()
    
    old = mx; m.append(mx_k); w.append(mx_w)

Ensemble Pearson = 0.8310 by beginning with model 4
Model 4 is oof_2.01_full.csv

Searching for best model to add... 
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 
Ensemble pearson = 0.8440 after adding model 0 with weight 0.435. Increase of 0.0129
Model 0 is oof_1.01_full.csv

Searching for best model to add... 
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 
Ensemble pearson = 0.8459 after adding model 9 with weight 0.205. Increase of 0.0019
Model 9 is oof_6.09_full.csv

Searching for best model to add... 
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 
Ensemble pearson = 0.8472 after adding model 8 with weight 0.195. Increase of 0.0013
Model 8 is oof_5.02_full.csv

Searching for best model to add... 
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 
Ensemble pearson = 0.8479 after adding model 1 with weight 0.390. Increase of 0.0006
Model 1 is oof_1.07_full.csv

Searching for best model to add... 
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 
Ensemble pearson = 0.8483 after adding model 5 with weight 0.090. Increas